In [1]:
!pip install numpy
!pip install spacy
!pip intall torch

     |████████████████████████████████| 16.8 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 635 kB 55.4 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 63 kB 4.9 MB/s s eta 0:00:01
     |████████████████████████████████| 76 kB 13.0 MB/s  eta 0:00:01
     |████████████████████████████████| 9.9 MB 55.4 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 103.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 100.4 MB/s eta 0:00:01
     |████████████████████████████████| 452 kB 90.6 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 114.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 17.7 MB/s  eta 0:00:01
     |████████████████████████████████| 149 kB 127.0 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 21.8 MB/s  eta 0:00:01
     |█████████████████████████

In [2]:
!pip install pandas
!pip install sklearn
!pip install nltk
!pip install spacy

     |████████████████████████████████| 11.7 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 96.4 MB/s eta 0:00:01
     |████████████████████████████████| 26.4 MB 3.0 MB/s eta 0:00:01     |███████████████████             | 15.8 MB 3.0 MB/s eta 0:00:04
     |████████████████████████████████| 39.8 MB 77.3 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 103.1 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=ecb0d0bf8dc3e06c28957277836805e586cd5b670e470b69e0a4fac7baff0f1e
  Stored in directory: /home/studio-lab-user/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
     |████████████████████████████████| 1.5 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 763 kB 23.1 MB/s eta 0:00:01


In [4]:
%conda install pytorch

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/studio-lab-user/.conda/envs/default

  added / updated specs:
    - pytorch


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    future-0.18.2              |   py39hf3d152e_4         715 KB  conda-forge
    libblas-3.9.0              |   13_linux64_mkl          13 KB  conda-forge
    libcblas-3.9.0             |   13_linux64_mkl          12 KB  conda-forge
    liblapack-3.9.0            |   13_linux64_mkl          12 KB  conda-forge
    libprotobuf-3.19.4         |       h780b84a_0         2.6 MB  conda-forge
    llvm-openmp-12.0.1         |       h4bd325d_1         2.8 

In [1]:
import numpy as np
import pandas as pd
import torch
import string
import statistics
import nltk
import spacy
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
class MODEL_EVAL_METRIC:
    accuracy = "accuracy"
    f1_score = "f1_score"

class Config:    
    EMB_SIZE = 300    
    NUM_FOLDS = 5
    NUM_EPOCHS = 20
    NUM_CV_RUN = 1
    MODEL_EVAL_METRIC = MODEL_EVAL_METRIC.accuracy

DATA_PATH = ""

In [3]:
df_train = pd.read_csv('train.csv',encoding= 'unicode_escape')
df_test = pd.read_csv('test.csv',encoding= 'unicode_escape')
print(f"Rows in train.csv = {len(df_train)}")
print(f"Rows in test.csv = {len(df_test)}")
pd.set_option('display.max_colwidth', None)
df_train.head()

Rows in train.csv = 18983
Rows in test.csv = 3263


,id,keyword,location,text,target
0,1.0,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4.0,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5.0,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6.0,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7.0,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [4]:
df_train_pos = df_train[df_train.target == 1]
df_train_neg = df_train[df_train.target == 0]
print(f"No. of positive training examples = {len(df_train_pos)}")
print(f"No. of negative training examples = {len(df_train_neg)}")
train_keywords_unique = df_train.keyword.unique()
print(f"No. of unique keywords = {len(train_keywords_unique)}")
df_train_notnull_keywords = df_train[~df_train.keyword.isnull()]
print(f"No of train examples with keyword not null = {len(df_train_notnull_keywords)}")

No. of positive training examples = 5385
No. of negative training examples = 13598
No. of unique keywords = 222
No of train examples with keyword not null = 18922


In [ ]:
train_tweet_vectors = None
test_tweet_vectors = None
nlp = spacy.load("en_core_web_lg")
with nlp.disable_pipes():
    train_tweet_vectors = np.array([nlp(row.text).vector for id, row in df_train.iterrows()])
    test_tweet_vectors = np.array([nlp(row.text).vector for id, row in df_test.iterrows()])
print(train_tweet_vectors.shape)    

In [ ]:
train_targets = df_train["target"]
vec_mean = train_tweet_vectors.mean(axis=0)
vec_std = train_tweet_vectors.std(axis=0)
print(vec_mean.shape, vec_std.shape)

In [ ]:
# for a training and label data in form of numpy arrays, return a fold_index array whose elements
# represent the fold index. The length of this fold_index array is same as length of input dataset
# and the items for which fold_index array value == cv iteration count are to be used for validation 
# in the corresponding cross validation iteration with rest of the items ( for which fold_index 
# array value != cv iteration count ) being used for training (typical ration being 80:20)
def get_skf_index(num_folds, X, y):
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = 42)
    train_fold_index = np.zeros(len(y))
    for fold, (train_index, val_index) in enumerate(skf.split(X=X, y=y)):
        train_fold_index[val_index] = [fold + 1] * len(val_index)
    return train_fold_index

k_folds = get_skf_index(num_folds=Config.NUM_FOLDS, X=train_tweet_vectors, y=train_targets)

In [ ]:
# Get the train and validation data for a specific fold. 
# X: numpy array of input features
# y: numpy array of target labels
# fold: fold index for which to create data loaders                                     
# kfolds: Array that marks each of the data items as belonging to a specific fold
def get_fold_data(fold, kfolds, X, y):
    fold += 1                         
    train_X = X[kfolds != fold]        
    train_y = y[kfolds != fold]    
    val_X = X[kfolds == fold]
    val_y = y[kfolds == fold]    
    return train_X, train_y, val_X, val_y

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def run_training(train_X, train_y, val_X, val_y, params):
    # Create the SVC model
    model = RandomForestClassifier(max_depth=2, random_state=0)
    # normalize the train and validation data 
    scaler = StandardScaler()
    train_X_scaled = scaler.fit_transform(train_X.astype(np.float32))
    val_X_scaled = scaler.fit_transform(val_X.astype(np.float32))
    model.fit(train_X_scaled, train_y.ravel())
    val_y_pred = model.predict(val_X_scaled)
    val_score = None
    if Config.MODEL_EVAL_METRIC == MODEL_EVAL_METRIC.accuracy:
        val_score = accuracy_score(val_y, val_y_pred)
    elif Config.MODEL_EVAL_METRIC == MODEL_EVAL_METRIC.f1_score:
        val_score = f1_score(val_y, val_y_pred)
    return round(val_score, 4), model
import tqdm

def cv_run(run, train_tweet_vectors, train_targets):
    fold_metrics_model = []
    params = {"C": 1.0, "kernel": "rbf"}
    for fold in tqdm.tqdm(range(Config.NUM_FOLDS)):        
        train_X, train_y, val_X, val_y = get_fold_data(fold, k_folds, train_tweet_vectors, train_targets)    
        fold_val_metric, fold_model = run_training(train_X, train_y, val_X, val_y, params)
        fold_metrics_model.append((fold_val_metric, fold_model))
    fold_metrics = [item[0] for item in fold_metrics_model]
    cv_metric_mean = statistics.mean(fold_metrics)
    cv_metric_std = statistics.stdev(fold_metrics)
    print(f"For cv run {run} fold_metrics = {fold_metrics}")
    print(f"mean val metric across folds = {cv_metric_mean}, val metric stdev across folds = {cv_metric_std}")
    return fold_metrics_model

In [25]:
run_fold_metrics_model = []
train_tweet_vectors = train_tweet_vectors - vec_mean
for run in tqdm.tqdm(range(Config.NUM_CV_RUN)):
    fold_metrics_model = cv_run(run+1, train_tweet_vectors, train_targets)    
    run_fold_metrics_model.append(fold_metrics_model)  

100%|██████████| 1/1 [02:25<00:00, 145.35s/it]

For cv run 1 fold_metrics = [0.8594, 0.8512, 0.848, 0.8656, 0.8559]
mean val metric across folds = 0.85602, val metric stdev across folds = 0.006905939472656886


In [26]:
fold_metrics_model = run_fold_metrics_model[0]
fold_metrics_model_sorted = sorted(fold_metrics_model, key=lambda x:x[0], reverse=True)    
best_cv_model = fold_metrics_model_sorted[0][1]

def test_predictions(model, test_tweet_vectors):
    scaler = StandardScaler()
    test_tweet_vectors_scaled = scaler.fit_transform(test_tweet_vectors.astype(np.float32))
    predictions = model.predict(test_tweet_vectors_scaled)
    print(f"Completed prediction for {len(predictions)} test rows")
    df_submission = pd.read_csv(DATA_PATH + 'sample_submission.csv')
    df_submission['target']= predictions
    df_submission.to_csv('submission_svc.csv',index=False)
    
#test_predictions(best_cv_model, test_tweet_vectors)   

In [28]:
params = {"C": 1.0, "kernel": "rbf"}
full_model = SVC(gamma='scale', C=params["C"], kernel=params["kernel"])
# normalize the train and validation data 
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_tweet_vectors.astype(np.float32))
full_model.fit(train_X_scaled, train_targets.ravel())
test_predictions(full_model, test_tweet_vectors)

Completed prediction for 3263 test rows
